# Traditional Word Embedding

In [1]:
# import packages
from gensim.models import word2vec
from monpa import utils
from tqdm import tqdm
import codecs
import logging
import monpa
import numpy as np
import os
import pandas as pd
import spacy

os.getcwd()

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


'D:\\Windows_Storage\\Storage\\Github\\KnowledgeGraph\\scripts'

# Configurations

In [2]:
selected_ratio = 0.5
data_entity = pd.read_csv("../results/210408_result/210408_dataset_entity_result_MONPA.csv", sep=",", encoding="utf8")
data_relation = pd.read_csv("../results/210408_result/210408_dataset_relation_result_MONPA.csv", sep=",", encoding="utf8")
output_token_for_word2vec = "../results/210408_result/210408_token_sentences_for_word2vec.csv"
model_save_path = "../models/210408_word2vec.model"

In [11]:
# MONPA
relation_dependencies_possible_list = []
relation_pos_possible_list = ["VH", "VC", "VJ", "VA"]
relation_pos_re = "^[V]"
entity_dependencies_possible_list = []
entity_pos_possible_list = ["Na", "Nv", "Neu", "Nes", "Nf", "Ng", "Nh", "Neqa", "Nep", "Ncd", "FW", "DE"]
entity_pos_re = "^[N]"
bracket_pos_list = ["PARENTHESISCATEGORY"]
punct_pos_list = ["COMMACATEGORY", "PERIODCATEGORY"]

# spaCy
# relation_dependencies_possible_list = ["ROOT", "nmod:prep", "prep", "agent", ]
# relation_pos_possible_list = ["VERB"]
# relation_pos_re = "^[V]"
# entity_dependencies_possible_list = ["compound:nn", "nsubj", "dep", "dobj"]
# entity_pos_possible_list = ["NOUN", "PROPN"]
# entity_pos_re = "^[N]"
# bracket_pos_list = []
# punct_pos_list = ["PUNCT"]

# Common, usually entity
sentences_splitter = ["，", "。", "！", "!", "？", "?"]
bracket_entity_list = ["(", ")", "（", "）"]
punct_entity_list = [" " * i for i in range(0, 100)]
conjuction_entity_list = ["的", "、", "之", "及", "與"]
not_entity_relation_list = ["的", "、", "之", "及", "與", "\r\n \r\n ", "\r\n \r\n  "] +\
[" " * i for i in range(0, 100)] + ["\n" * i for i in range(0, 100)]

# Import Data

In [3]:
textElement = ""

for fileIndex, fileElement in enumerate(tqdm(os.listdir("../data/"))):
    file = codecs.open("../data/" + fileElement, 'r', encoding='utf8', errors='ignore')
    for textIndex, textLines in enumerate(file):
        textElement += textLines
        
    
print("Text Length: ", len(textElement), " \n\n", textElement[-100:])

100%|██████████| 15/15 [00:00<00:00, 202.71it/s]

Text Length:  207715  

 管中流過，凝結下來之蒸

氣附著管外殼上。蒸氣可在管外凝結成一層薄膜後，再排

至儲存槽，或排出後做適當之處置。在接觸式冷凝器中，

則噴灑冷的液體以冷凝廢氣中之揮發性成份。 

5 -30 





# Import stopwords list

In [4]:
stopword_list = []

for fileIndex, fileElement in enumerate(os.listdir("./stopwords/")):
    if fileElement[-2:] != "md":
        data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")
        stopword_list += data_temp.iloc[:, 0].tolist()
        
stopword_list = list(dict.fromkeys(stopword_list))
        
print(len(stopword_list), "\n\n", stopword_list[-50:])     


3249 

 ['where', 'whereafter', 'whereas', 'whereby', 'wherein', "where's", 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', "who's", 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', 'wonder', "won't", 'would', "wouldn't", 'yes', 'yet', 'you', "you'd", "you'll", 'your', "you're", 'yours', 'yourself', 'yourselves', "you've", 'zero', 'zt', 'ZT', 'zz', 'ZZ', '準備', '覆雜', '心裏', '註意', '裏面']


<ipython-input-4-caa2e0aa211a>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")


# Create User-Defined Dict by entities

In [5]:
print(int(np.around(len(data_entity) * selected_ratio, decimals=0)), int(np.around(len(data_relation) * selected_ratio, decimals=0)))
data_dict = pd.concat([data_entity.iloc[:int(np.around(len(data_entity) * selected_ratio, decimals=0)),:],\
                    data_relation.iloc[:int(np.around(len(data_relation) * selected_ratio, decimals=0)),:]], ignore_index=True)
print(len(data_dict))

entity_list = data_dict.iloc[:, 0].tolist()

print(entity_list[:10], entity_list[-10:])

# remove blank inside entity element
for elementIndex, element in enumerate(entity_list):
    entity_list[elementIndex] = str(entity_list[elementIndex]).replace(" ", "")
    entity_list[elementIndex] = str(entity_list[elementIndex]).replace("  ", "")
    entity_list[elementIndex] = str(entity_list[elementIndex]).replace("\"", "")

for i in range(len(entity_list)):
    if type(data_dict.loc[i, "Ratio"]) == pd.Series:
        print(data_dict.loc[i, "Ratio"])
    
    
data_dict_output = pd.DataFrame({
    "0": entity_list,
    "1": [int(1000000000 * data_dict.loc[i, "Ratio"]) for i in range(len(entity_list))],
    "2": ["UserEntity" for i in range(len(entity_list))],
})

data_dict_output.to_csv("./dicts/monpa_entity_dict.txt", encoding="utf8", sep=" ", index=None, header=None)

4291 1337
5628
['晶圓', '技術', '製程', '晶片', '產品', '表面', '成本', '設備', '市場', '測品'] ['力球', '類機', '類表', '漏油', '切出', '逸散', '逸 出', '精度', '精確', '見下']


# Segment data 

In [6]:
# read in entity dictionary

monpa.load_userdict("./dicts/monpa_entity_dict.txt")

In [12]:
# divide sentences and segment by monpa
textList = utils.short_sentence(textElement)

cutResultList = []

for textIndex, textElement in enumerate(tqdm(textList)):
    cutResult = monpa.pseg(textElement)
    for elementIndex, element in enumerate(cutResult):
        if element in stopword_list:
            cutResult[elementIndex] = ""
            
        
    cutResultList.append(cutResult)

print(cutResultList[:5])

100%|██████████| 1/1 [00:00<00:00,  4.76it/s]

[[('蒸氣', 'UserEntity'), ('可', 'D'), ('在', 'P'), ('管', 'VC'), ('外', 'Ng'), ('凝結', 'UserEntity'), ('成', 'UserEntity'), ('一層', 'UserEntity'), ('薄膜', 'UserEntity'), ('後', 'Ng'), ('，', 'COMMACATEGORY'), ('再', 'D'), ('排', 'VC'), ('  至', 'PER'), ('儲存', 'UserEntity'), ('槽', 'Na'), ('，', 'COMMACATEGORY'), ('或', 'Caa'), ('排出', 'UserEntity'), ('後', 'Ng'), ('做', 'VC'), ('適當', 'VH'), ('之', 'Nh'), ('處置', 'UserEntity'), ('。', 'PERIODCATEGORY'), ('在', 'P'), ('接觸', 'UserEntity'), ('式', 'Na'), ('冷凝', 'UserEntity'), ('器', 'Na'), ('中', 'Ng'), ('，', 'COMMACATEGORY'), (' ', 'FW'), (' ', 'FW'), ('則', 'D'), ('噴灑', 'VC'), ('冷', 'FW'), ('的', 'T'), ('液體', 'UserEntity'), ('以', 'P'), ('冷凝', 'UserEntity'), ('廢氣', 'UserEntity'), ('中', 'Ng'), ('之', 'DE'), ('揮發', 'UserEntity'), ('性', 'Na'), ('成', 'UserEntity'), ('份', 'Na'), ('。', 'PERIODCATEGORY'), (' ', 'FW'), (' ', 'FW'), (' ', 'FW'), ('5', 'Neu'), (' ', 'FW'), ('-', 'DASHCATEGORY'), ('30', 'Neu')]]


In [8]:
# create data for word2vec training

lineList = []

for sentencesIndex, sentencesElementList in enumerate(tqdm(cutResultList)):
    tokenElement = ""
    for elementIndex, element in enumerate(sentencesElementList):
        if element not in stopword_list:
            tokenElement += (element + " ")
    lineList.append(tokenElement)
    
data_output_word2vec = pd.DataFrame({
    "Token Sentence":lineList
})

data_output_word2vec.to_csv(output_token_for_word2vec, encoding="utf8", sep=" ", header=None, index=None)

100%|██████████| 1332/1332 [00:06<00:00, 219.88it/s]


# Word2Vec Model Training
* Reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py
* Reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_annoy.html

In [9]:
# import packages
from gensim.models import word2vec
from monpa import utils
from tqdm import tqdm
import codecs
import logging
import monpa
import numpy as np
import os
import pandas as pd
import spacy

os.getcwd()

'D:\\Windows_Storage\\Storage\\Github\\KnowledgeGraph\\scripts'

In [10]:
sentences = word2vec.LineSentence(output_token_for_word2vec)
model = word2vec.Word2Vec(sentences, size=240)

model.save(model_save_path)

# Some opertation needed but would make kernel down
from gensim.models import word2vec

model = word2vec.Word2Vec.load("../models/210408_word2vec.model")

print(model.wv.similarity("晶圓", "價值"))
print(model.wv.most_similar(["晶圓", "矽"]))